In [ ]:
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np
import os
import numpy as np
import bigfish
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.multistack as multistack
import bigfish.plot as plot
from sklearn.cluster import DBSCAN
import pickle
import glob
from scipy.spatial import ConvexHull

# Read in and pad all MERSCOPE smFISH images

In [ ]:
path_to_imgs = r'D:\Tatsuya\merscope\202210221316_avrrpt2-9h-rep1_VMSC01101\out\images'

In [ ]:
file_path = os.path.join(path_to_imgs, 'mosaic_BacMeta_z3.tif')
an_image_bac = io.imread(file_path)
bac_image_padded = np.pad(an_image_bac, pad_width=((0, 2000 - np.shape(an_image_bac)[0]%2000), (0,0)))
bac_image_padded = np.pad(bac_image_padded, pad_width=((0, 0), (0,2000 - np.shape(an_image_bac)[1]%2000)))

In [ ]:
file_path =  os.path.join(path_to_imgs, 'mosaic_DAPI_z3.tif')
an_image_dapi = io.imread(file_path)
dapi_image_padded = np.pad(an_image_dapi, pad_width=((0, 2000 - np.shape(an_image_dapi)[0]%2000), (0,0)))
dapi_image_padded = np.pad(dapi_image_padded, pad_width=((0, 0), (0,2000 - np.shape(an_image_dapi)[1]%2000)))

In [ ]:
file_path = os.path.join(path_to_imgs, 'mosaic_sid_z3.tif')
an_image_sid = io.imread(file_path)
sid_image_padded = np.pad(an_image_sid, pad_width=((0, 2000 - np.shape(an_image_sid)[0]%2000), (0,0)))
sid_image_padded = np.pad(sid_image_padded, pad_width=((0, 0), (0,2000 - np.shape(an_image_sid)[1]%2000)))

In [ ]:
file_path = os.path.join(path_to_imgs, 'mosaic_hrpl_z3.tif')
an_image_hrpl = io.imread(file_path)
hrpl_image_padded = np.pad(an_image_hrpl, pad_width=((0, 2000 - np.shape(an_image_hrpl)[0]%2000), (0,0)))
hrpl_image_padded = np.pad(hrpl_image_padded, pad_width=((0, 0), (0,2000 - np.shape(an_image_hrpl)[1]%2000)))

In [ ]:
file_path = os.path.join(path_to_imgs, 'mosaic_pvds_z3.tif')
an_image_pvds = io.imread(file_path)
pvds_image_padded = np.pad(an_image_pvds, pad_width=((0, 2000 - np.shape(an_image_pvds)[0]%2000), (0,0)))
pvds_image_padded = np.pad(pvds_image_padded, pad_width=((0, 0), (0,2000 - np.shape(an_image_pvds)[1]%2000)))

In [ ]:
output_folder = r'D:\Alex\MERSCOPE_reanalysis_output\smFISH_output\avr_9hr'
os.mkdir(output_folder)

# Quantify spots across all images (Hyperparameters will need tuning if using a different set of images)

Everything will get saved to the outfolder. Read BigFISH documentation to choose the correct hyperparameters.

In [ ]:
from typing import overload
from sklearn import cluster
from tqdm.notebook import tqdm

over_counter = 0
for i_o in range(2000, len(bac_image_padded[0]), 2000):
    for j_o in range(2000, len(bac_image_padded), 2000): 
        iamge_play = bac_image_padded[i_o-2000:i_o, j_o-2000:j_o]
        iamge_play_dapi = dapi_image_padded[i_o-2000:i_o, j_o-2000:j_o]
        iamge_play_sid = sid_image_padded[i_o-2000:i_o, j_o-2000:j_o]
        iamge_play_hrpl = hrpl_image_padded[i_o-2000:i_o, j_o-2000:j_o]
        iamge_play_pvds = pvds_image_padded[i_o-2000:i_o, j_o-2000:j_o]
#         spots, threshold = detection.detect_spots(  
#             images=iamge_play, 
#             threshold=95,
#             return_threshold=True, 
#             spot_radius=(17, 17),
#             voxel_size = (3, 3))   
        spots, threshold = detection.detect_spots(  
            images=iamge_play, 
            threshold=200,
            return_threshold=True, 
            log_kernel_size=(1.456, 1.456),
            minimum_distance=(1.456, 1.456))
        spots_dapi = np.array([])
        spots_sid = np.array([])
        spots_hrpl = np.array([])
        spots_pvds = np.array([])
        spots_dapi, threshold_dapi = detection.detect_spots(  
            images=iamge_play_dapi, 
            return_threshold=True, 
            threshold=180,
            log_kernel_size=(1.456, 1.456),
            minimum_distance=(1.456, 1.456))

        spots_sid, threshold_sid = detection.detect_spots(  
            images=iamge_play_sid, 
            return_threshold=True,
            threshold=1000,
            log_kernel_size=(1.456, 1.456),
            minimum_distance=(1.456, 1.456))
        
        spots_hrpl, threshold_sid = detection.detect_spots(  
            images=iamge_play_hrpl, 
            return_threshold=True,
            threshold=200,
            log_kernel_size=(1.456, 1.456),
            minimum_distance=(1.456, 1.456))

        spots_pvds, threshold_sid = detection.detect_spots(  
            images=iamge_play_pvds, 
            return_threshold=True,
            threshold=300,
            log_kernel_size=(1.456, 1.456),
            minimum_distance=(1.456, 1.456))

        total_spots = list(spots_dapi)
        type_spots = ['dapi' for i in range(len(spots_dapi))]
        for i in spots:
            total_spots.append(i)
            type_spots.append('bac')
        for i in spots_sid:
            total_spots.append(i)
            type_spots.append('sid')

        total_spots = np.array(total_spots)
        if len(total_spots) > 0:
            clustering = DBSCAN(eps = 35, min_samples=5).fit(total_spots)
            cluster_types = np.unique(clustering.labels_)
            if len(cluster_types) != 1:
                for i in cluster_types:
                    if i != -1:
                        locations_cluster = np.where(clustering.labels_ == i)
                        counts_labels = np.unique(np.array(type_spots)[locations_cluster], return_counts=True)
                        counts_dapi = counts_labels[1][np.where(counts_labels[0] == 'dapi')]
                        counts_bac = counts_labels[1][np.where(counts_labels[0] == 'bac')]
                        counts_sid = counts_labels[1][np.where(counts_labels[0] == 'sid')]
                        if len(counts_bac) == 0:
                            counts_bac = np.array([0])
                        if len(counts_dapi) == 0:
                            counts_dapi = np.array([0])
                        if len(counts_sid) == 0:
                            counts_sid = np.array([0])
                        if (counts_dapi+counts_sid)/(counts_dapi+counts_sid+counts_bac) > 0.3:
                            clustering.labels_[locations_cluster] = -1

            cl_new, index_types = np.unique(clustering.labels_, return_index=True)
            import numpy as np
            import scipy as sp

            def dbscan_predict(dbscan_model, X_new, metric=sp.spatial.distance.cosine):
                # Result is noise by default
                y_new = np.ones(shape=len(X_new), dtype=int)*-1 

                # Iterate all input samples for a label
                for j, x_new in enumerate(X_new):
                    # Find a core sample closer than EPS
                    for i, x_core in enumerate(dbscan_model.components_): 
                        if metric(x_new, x_core) < dbscan_model.eps:
                            # Assign label of x_core to x_new
                            y_new[j] = dbscan_model.labels_[dbscan_model.core_sample_indices_[i]]
                            break

                return y_new
            x_new = [i for i in spots_hrpl]
            for i in spots_pvds:
                x_new.append(i)
            figure, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 10))
            ax1.imshow(iamge_play)
            ax1.scatter([i[1] for i in spots], [i[0] for i in spots], c='red', s=1, label = 'Bacterial Genes')
            ax1.scatter([i[1] for i in spots_dapi], [i[0] for i in spots_dapi], c='yellow', s=1, label = 'DAPI')
            ax1.scatter([i[1] for i in spots_sid], [i[0] for i in spots_sid], c='green', s=1, label = 'SID')
            ax1.scatter([i[1] for i in spots_hrpl], [i[0] for i in spots_hrpl], c='blue', s=1, label = 'hrpL')
            ax1.scatter([i[1] for i in spots_pvds], [i[0] for i in spots_pvds], c='purple', s=1, label = 'pvds')
            ax1.set_title('Bacterial Genes and Dapi Overlay')
            ax1.axis('off')
            ax2.imshow(iamge_play)
            ax2.scatter([i[1] for i in total_spots], [i[0] for i in total_spots], c=clustering.labels_, s=1)
            
            
            ax3.imshow(iamge_play_dapi)
            ax3.set_title('SID image')
            ax3.axis('off')
            for i in range(len(cl_new)):
                if cl_new[i] != -1:
                    ax2.annotate(cl_new[i], (total_spots[index_types[i]][1], total_spots[index_types[i]][0]), color='white')
            ax2.set_title('Candidate Bacterial Genes')
            ax2.axis('off')
            try:
                os.mkdir(os.path.join(output_folder, 'pics'))
                os.mkdir(os.path.join(output_folder, 'total_spots'))
                os.mkdir(os.path.join(output_folder, 'type_spots'))
                os.mkdir(os.path.join(output_folder, 'hrpl_spots'))
                os.mkdir(os.path.join(output_folder, 'pvds_spots'))
                os.mkdir(os.path.join(output_folder, 'dbscan'))
            except:
                None
            plt.savefig(os.path.join(output_folder, 'pics', 'pic'+str(i_o)+'_'+str(j_o)+'.png'))
            plt.close()
        
            np.save(os.path.join(output_folder, 'total_spots', 'total_spots'+str(i_o)+'_'+str(j_o)+'.npy'), total_spots)
            np.save(os.path.join(output_folder, 'type_spots', 'type_spots'+str(i_o)+'_'+str(j_o)+'.npy'), type_spots)
            np.save(os.path.join(output_folder, 'hrpl_spots', 'hrpl_spots'+str(i_o)+'_'+str(j_o)+'.npy'), spots_hrpl)
            np.save(os.path.join(output_folder, 'pvds_spots', 'pvds_spots'+str(i_o)+'_'+str(j_o)+'.npy'), spots_pvds)
            
            file = open(os.path.join(output_folder, 'dbscan', 'dbscan'+str(i_o)+'_'+str(j_o)+'.pkl'), 'wb')
            # dump information to that file
            pickle.dump(clustering, file)
            file.close()


        over_counter += 1
        print(over_counter)

# Use DBscan to find bacterial colonies

In [ ]:
import glob
path = output_folder
dbscans = glob.glob(os.path.join(path, 'dbscan', '*'))

In [ ]:


fov = []
total_centroids = []
total_cluster_components = []
total_labels = []
for db in dbscans:
    with open(db, 'rb') as f:
        db_contents = pickle.load(f)
    f.close()
    total_spots_file = db.replace('dbscan', 'total_spots').replace('.pkl', '.npy')
    total_spots_array = np.load(total_spots_file)
    fov.append(os.path.basename(db))
    centroids = []
    all_components = []
    for lab in np.unique(db_contents.labels_):
        if lab != -1:
            components_cluster = total_spots_array[np.where(db_contents.labels_ == lab)]
            centroid = np.mean(components_cluster, axis = 0)
            centroids.append(centroid)
            all_components.append(components_cluster)
    total_centroids.append(centroids)
    total_cluster_components.append(all_components)
    total_labels.append(db_contents.labels_)


In [ ]:
fov_nums = [np.array(i.replace('dbscan', '').replace('.pkl', '').split('_')).astype(int) for i in fov]

In [ ]:
xval_positions = []
yval_positions = []
for i in range(len(total_centroids)):
    xval_cent = []
    yval_cent = []
    if len(total_centroids[i]) > 0:
        for j in range(len(total_centroids[i])):
            xval_cent.append(fov_nums[i][0] - 2000 + total_centroids[i][j][0])
            yval_cent.append(fov_nums[i][1] - 2000 + total_centroids[i][j][1])
    xval_positions.append(np.array(xval_cent))
    yval_positions.append(np.array(yval_cent))

# Exclude spots detected on the edge of imaging plane by making box of exclusion

In [ ]:
def create_exclusion_box(xvals, yvals, top, bottom, left, right):
    return xvals[~((xvals > left) & (xvals < right) & (yvals > bottom) & (yvals < top))], yvals[~((xvals > left) & (xvals < right) & (yvals > bottom) & (yvals < top))]

In [ ]:
xcoords = np.array([])
for pos in xval_positions:
    xcoords = np.append(xcoords, pos)
ycoords = np.array([])
for pos in yval_positions:
    ycoords = np.append(ycoords, pos)

In [ ]:
try_xval, try_yval = xcoords, ycoords

In [ ]:
#9hr exclusion
try_xval, try_yval = create_exclusion_box(xcoords, ycoords, top = 3000, bottom = 0, left = 0, right =5000)
try_xval, try_yval = create_exclusion_box(try_xval, try_yval, top = 4000, bottom = 0, left = 0, right =100000)

# Enter yes or no on candidate images as manual filtering

In [ ]:
final_spots_after_filtering = []
scatters = np.unique(np.array([try_xval, try_yval]).T, axis = 0)
for img in scatters:
    iamge_play = bac_image_padded[int(img[0]-2000):int(img[0]+2000), int(img[1]-2000):int(img[1]+2000)]
    dap_iamge_play = dapi_image_padded[int(img[0]-2000):int(img[0]+2000), int(img[1]-2000):int(img[1]+2000)]
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.imshow(iamge_play)
    ax2.imshow(dap_iamge_play)
    plt.show()
    answer = input()
    if answer == 'n':
        continue
    else:
        try:
            os.mkdir(os.path.join(path, 'final_spot_images'))
        except:
            None
        plt.savefig(os.path.join(path, 'final_spot_images', str(int(img[0]))+'_'+str(int(img[1]))+'.png'))
        final_spots_after_filtering.append((img[0], img[1]))
    plt.close()


# Save final candidates

In [ ]:
centroids_final_spots = []
for first in final_spots_after_filtering:
    for second in range(len(xval_positions)):
        if (first[0] in xval_positions[second]) and (first[1] in yval_positions[second]):
            centroids_final_spots.append((first, second))
            

In [ ]:
np.save(os.path.join(path, 'centroids_final_spots.npy'),centroids_final_spots)

# Higher Resolution Quantification and Filtering. Tune Hyperparameters accordingly

In [ ]:
from re import S
import math
class Found(Exception): pass
hull_volumes = []
hull_points = []
new_centroids_final_spots = []
for img in centroids_final_spots:
        window = 300
        iamge_play = bac_image_padded[int(img[0][0]-window):int(img[0][0]+window), int(img[0][1]-window):int(img[0][1]+window)]
        hrpl_iamge_play = hrpl_image_padded[int(img[0][0]-window):int(img[0][0]+window), int(img[0][1]-window):int(img[0][1]+window)]
        pvds_iamge_play = pvds_image_padded[int(img[0][0]-window):int(img[0][0]+window), int(img[0][1]-window):int(img[0][1]+window)]
        spots, threshold = detection.detect_spots(  
                images=iamge_play, 
                threshold=90,
                return_threshold=True, 
                log_kernel_size=(1.456, 1.456),
                minimum_distance=(1.456, 1.456))
        real_spots, threshold_bac = detection.detect_spots(  
                images=iamge_play, 
                threshold=95,
                return_threshold=True, 
                spot_radius=(17, 17),
                voxel_size = (3, 3))   
        if len(real_spots) == 0:
                real_spots = [[150, 150]]
        spots_hrpl, threshold_sid = detection.detect_spots(  
                images=hrpl_iamge_play, 
                return_threshold=True,
                threshold=200,
                spot_radius=(6, 4),
                voxel_size = (3, 3))   

        spots_pvds, threshold_sid = detection.detect_spots(  
                images=pvds_iamge_play, 
                return_threshold=True,
                threshold=115,
                spot_radius=(6, 6),
                voxel_size = (3, 3)) 

        total_spots_second = list(spots)
        type_spots = ['bac' for i in range(len(spots))]
        for i in spots_pvds:
                total_spots_second.append(i)
                type_spots.append('pvds')
        for i in spots_hrpl:
                total_spots_second.append(i)
                type_spots.append('hrpl')
        for i in real_spots:
                total_spots_second.append(i)
                type_spots.append('bac_whole')

        total_spots_second = np.array(total_spots_second)
        
        try_samples = [30, 10, 5, 3]
        use_cluster = []
        ct_try = 0
        try:
                while True:
                        clustering_second = DBSCAN(eps = 100, min_samples=try_samples[ct_try]).fit(total_spots_second)
                        ct_try += 1
                        for uniq in np.unique(clustering_second.labels_):
                            if not ((len(np.unique(clustering_second.labels_)) == 1) and (-1 in clustering_second.labels_)):
                                if uniq != -1:
                                    point_labels = np.array([np.array(type_spots)[f] for f in np.where(clustering_second.labels_ == uniq)]).squeeze()
                                    #print(len([i for i in point_labels if i == 'bac']))
                                    try:
                                            len([i for i in point_labels if i == 'bac'])
                                    except:
                                            None
                                    if len([i for i in point_labels if i == 'bac']) < 3:
                                            None
                                    else:
                                            use_cluster.append(uniq)
                                if len(use_cluster) > 0:
                                        raise Found
        except Found:
                print('found')

        

        hull_list = []
        point_list = []
        point_labels = []
        for uniq in use_cluster:
                if uniq != -1:
                        points = np.array([total_spots_second[f] for f in np.where(clustering_second.labels_ == uniq)]).squeeze()
                        points = np.array([np.array([i[1], i[0]]) for i in points])
                        point_labels.append(np.array([np.array(type_spots)[f] for f in np.where(clustering_second.labels_ == uniq)]).squeeze())
                        hull = ConvexHull(points)
                        point_list.append(points)
                        hull_list.append(hull)
        from_center = []
        for j in point_list:
                average_points = np.mean(j, axis = 0)
                distance_po = math.sqrt(((average_points[1] - 150)**2)+((average_points[0] - 150)**2))
                from_center.append(distance_po)
        try:
                hull = hull_list[np.where(from_center == np.min(from_center))[0][0]]
                points = point_list[np.where(from_center == np.min(from_center))[0][0]]
                point_labels = point_labels[np.where(from_center == np.min(from_center))[0][0]]
        except:
                print(use_cluster)
                print(point_list)
                continue


        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (10, 10), dpi = 200)

        
        ax1.imshow(iamge_play)
        ax1.scatter([i[1] for i in real_spots], [i[0] for i in real_spots], c='red', s=1, label = 'Bacterial Genes')
        
        
        ax1.set_title('Bacterial Colony Identification')
        ax1.legend(loc=2, prop={'size': 6})
        #ax1.plot(points[:,0], points[:,1], 'o')
        for simplex in hull.simplices:
                ax1.plot(points[simplex, 0], points[simplex, 1], 'w')
        hull_volumes.append(hull.volume)
        hull_points.append((points, total_spots_second, type_spots))
        ax1.plot(points[hull.vertices,0], points[hull.vertices,1], 'w', lw=0.5)
        #ax1.plot(points[hull.vertices[0],0], points[hull.vertices[0],1], 'ro', lw= 0.5)
        ax2.imshow(hrpl_iamge_play)
        ax2.scatter([i[1] for i in spots_hrpl], [i[0] for i in spots_hrpl], c='red', s=1, label = 'hrpL')
        ax2.set_title('hrpL Raw')
        ax3.imshow(pvds_iamge_play)
        ax3.scatter([i[1] for i in spots_pvds], [i[0] for i in spots_pvds], c='red', s=1, label = 'pvds')
        ax3.set_title('pvds Raw')
        fig1 = plt.gcf()
        plt.show()
        keep = input()
        try:
                os.mkdir(os.path.join(path, 'final_spot_for_adata'))
                os.mkdir(os.path.join(path, 'final_spot_for_adata','final_spot_images'))
                os.mkdir(os.path.join(path, 'final_spot_for_adata', 'point_labels'))
                os.mkdir(os.path.join(path, 'final_spot_for_adata', 'points'))
                os.mkdir(os.path.join(path, 'final_spot_for_adata', 'hulls'))
        except:
                None
        if keep == 'y':
                fig1.savefig(os.path.join(path, 'final_spot_for_adata','final_spot_images', 'Bac_cluster_'+str(int(img[0][0]))+'_'+str(int(img[0][1]))+'.png'))
                np.save(os.path.join(path, 'final_spot_for_adata','point_labels', 'Bac_cluster_point_labels_'+str(int(img[0][0]))+'_'+str(int(img[0][1]))+'.npy'), point_labels)
                np.save(os.path.join(path, 'final_spot_for_adata','points', 'Bac_cluster_points_'+str(int(img[0][0]))+'_'+str(int(img[0][1]))+'.npy'), points)
                file = open(os.path.join(path, 'final_spot_for_adata','hulls', 'Bac_cluster_hulls_'+str(int(img[0][0]))+'_'+str(int(img[0][1]))+'.pkl'), 'wb')
                # dump information to that file
                pickle.dump(hull, file)
                file.close()        
                new_centroids_final_spots.append(img)
                
                plt.close()

In [ ]:
centroid_adata, median_distance_from_centroid, number_hrpl = [], [], []
hrpl_median_distance_from_centroid, number_pvds, pvds_median_distance_from_centroid = [], [], []
total_spots_in_colony, hull_area = [], []

# Convert to microns from pixels

In [ ]:
import transform_to_microns
mer_images = r'D:\Tatsuya\merscope\202210221316_avrrpt2-9h-rep1_VMSC01101\out\images'
transformer = transform_to_microns.MerfishTransform(os.path.join(mer_images, 'micron_to_mosaic_pixel_transform.csv'))


# Calculate statistics about each colony

In [ ]:
files_hull = glob.glob(os.path.join(path, 'final_spot_for_adata', 'points', '*'))
for i in files_hull:
    point_array = np.load(i)
    point_labels_array = np.load(i.replace('points', 'point_labels'))
    total_spots_in_colony.append(len([j for j in point_labels_array if j != 'bac']))

    average_points = np.mean(point_array, axis = 0)

    avg_array = []
    for j in point_array:
        distance_po = math.sqrt(((average_points[1] - j[1])**2)+((average_points[0] - j[0])**2))
        avg_array.append(distance_po)
    median_distance_from_centroid.append(np.mean(avg_array))
    
    with open(i.replace('points', 'hulls').replace('.npy', '.pkl'), 'rb') as f:
        huller = pickle.load(f)
    f.close()
    hull_area.append(huller.volume)

    number_hrpl.append(len([j for j in point_labels_array if j == 'hrpl']))
    number_pvds.append(len([j for j in point_labels_array if j == 'pvds']))

    avg_array = []
    for j in range(len(point_array)):
        if point_labels_array[j] == 'hrpl':
            distance_po = math.sqrt(((average_points[1] - point_array[j][1])**2)+((average_points[0] - point_array[j][0])**2))
            avg_array.append(distance_po)
    try:
        hrpl_median_distance_from_centroid.append(np.mean(avg_array))
    except:
        hrpl_median_distance_from_centroid.append(0)
    
    avg_array = []
    for j in range(len(point_array)):
        if point_labels_array[j] == 'pvds':
            distance_po = math.sqrt(((average_points[1] - point_array[j][1])**2)+((average_points[0] - point_array[j][0])**2))
            avg_array.append(distance_po)
    try:
        pvds_median_distance_from_centroid.append(np.mean(avg_array))
    except:
        pvds_median_distance_from_centroid.append(0)   

In [ ]:
centroid_adata = new_centroids_final_spots

In [ ]:
new_X = np.array([[cen[0][0] for cen in centroid_adata],[cen[0][1] for cen in centroid_adata]]).T


In [ ]:
np.save(r'D:\Alex\MERSCOPE_reanalysis_output\troubleshooting\out.npy', new_X)

In [ ]:

import transform_to_microns
mer_images = r'D:\Tatsuya\merscope\202210221316_avrrpt2-9h-rep1_VMSC01101\out\images'

transformer = transform_to_microns.MerfishTransform(os.path.join(mer_images,'micron_to_mosaic_pixel_transform.csv'))
np.save(r'D:\Alex\MERSCOPE_reanalysis_output\troubleshooting\transformed.npy', transformer.pixel_to_micron_transform(np.load(r'D:\Alex\MERSCOPE_reanalysis_output\troubleshooting\out.npy')))
coordinates_bacteria_spatial = np.load(r'D:\Alex\MERSCOPE_reanalysis_output\troubleshooting\transformed.npy')

# Create final bacterial adata and save final positions

In [ ]:
centroid_adata = coordinates_bacteria_spatial

In [ ]:
import anndata as ad
import scanpy as sc
import pandas as pd

In [ ]:
observations = pd.DataFrame(zip(median_distance_from_centroid, number_hrpl, hrpl_median_distance_from_centroid, number_pvds, pvds_median_distance_from_centroid, total_spots_in_colony, hull_area), columns = 'median_distance_from_centroid, number_hrpl, hrpl_median_distance_from_centroid, number_pvds, pvds_median_distance_from_centroid, total_spots_in_colony, hull_area'.split(', '))

In [ ]:
observations.to_csv(os.path.join(output_folder, 'observations.csv'))

In [ ]:
np.save(os.path.join(output_folder, 'centroids_final_spots.npy'), centroid_adata)